## Procesamiento de data

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_table('datos.txt', sep = ' ')
df.columns = [column.replace(" ", "_") for column in df.columns]
df

,id,x,y,z,tonn,blockvalue,destination,CU%,process_profit
0,0,11,0,18,16380,-12285.0,2,0.0,0.0
1,1,12,0,18,16380,-12285.0,2,0.0,0.0
2,2,13,0,18,16380,-12285.0,2,0.0,0.0
3,3,13,1,18,16380,-12285.0,2,0.0,0.0
4,4,14,0,18,16380,-12285.0,2,0.0,0.0
5,5,14,1,18,16380,-12285.0,2,0.0,0.0
6,6,15,1,18,16380,-12285.0,2,0.0,0.0
7,7,16,1,18,16380,-12285.0,2,0.0,0.0
8,8,7,0,17,16380,-12285.0,2,0.0,0.0
9,9,8,0,17,12000,-9000.0,2,0.0,0.0


In [22]:
df

,id,x,y,z,tonn,blockvalue,destination,CU%,process_profit
0,0,11,0,18,16380,-12285.0000,2,0.000,0.0000
1,1,12,0,18,16380,-12285.0000,2,0.000,0.0000
2,2,13,0,18,16380,-12285.0000,2,0.000,0.0000
3,3,13,1,18,16380,-12285.0000,2,0.000,0.0000
4,4,14,0,18,16380,-12285.0000,2,0.000,0.0000
...,...,...,...,...,...,...,...,...,...
14148,14148,60,22,0,16260,136481.0096,1,0.924,148676.0096
14149,14149,60,23,0,16260,132053.7237,1,0.908,144248.7237
14150,14150,60,24,0,16260,128043.1476,1,0.888,140238.1476
14151,14151,60,25,0,16260,117441.3854,1,0.836,129636.3854


In [40]:
i=0
combined_df = pd.DataFrame()
combined_dfm = pd.DataFrame()

while (combined_df.shape)[0]<1601:
    df_tem  = df.sample(n=1)
    df_tem = df_tem[['x', 'y']]
    df_tem_two = np.array(df_tem)
    A = df_tem_two[0][0]
    B = df_tem_two[0][1]
    df_tem_three = df.query('x == @A and y == @B', inplace=False)
    if (combined_df.shape)[0]<150:
        combined_dfm = pd.concat([combined_dfm, df_tem_three], ignore_index=True)
        combined_dfm = combined_dfm.drop_duplicates()
    combined_df = pd.concat([combined_df, df_tem_three], ignore_index=True)
    combined_df = combined_df.drop_duplicates()
    i=i+1

In [41]:
combined_df

,id,x,y,z,tonn,blockvalue,destination,CU%,process_profit
0,1268,44,22,15,16380,-12285.00000,2,0.000,0.00000
1,2554,44,22,14,15360,-11520.00000,2,0.000,0.00000
2,4035,44,22,13,15360,-11520.00000,2,0.000,0.00000
3,5677,44,22,12,15360,-11520.00000,2,0.000,0.00000
4,7309,44,22,11,15360,-11520.00000,2,0.000,0.00000
5,8762,44,22,10,15360,-11520.00000,2,0.000,0.00000
6,10044,44,22,9,15360,-11520.00000,2,0.000,0.00000
7,11154,44,22,8,15360,-11520.00000,2,0.000,0.00000
8,11935,44,22,7,16140,115495.14150,1,0.800,127600.14150
9,12557,44,22,6,16140,101600.84610,1,0.746,113705.84610


In [161]:
x = np.array(combined_df['x']).astype(float)
y = np.array(combined_df['y']).astype(float)
z = np.array(combined_df['z']).astype(float)
cu_value = np.array(combined_df['CU%']).astype(float)

## Gstools

In [5]:
import gstools as gs

In [102]:
# Crear un modelo de variograma esférico
model = gs.Spherical(dim=3, var=0.5, len_scale=1.5)

# Crear una instancia de Kriging Ordinario con el modelo
krige = gs.krige.Ordinary(model, [x, y, z], cu_value,exact=True)


In [50]:
# Realizar el Kriging en la grilla deseada
gridx = np.linspace(3, x.max(), 10)
gridy = np.linspace(2, y.max(), 10)
gridz = np.linspace(3, z.max(), 10)
# gridx, gridy, gridz = np.meshgrid(gridx, gridy, gridz)

# Obtener los valores estimados y las varianzas
kriged_field, variances = krige([gridx, gridy, gridz])

In [51]:
kriged_field

array([0.37945997, 0.37945997, 0.37945997, 0.39833957, 0.37957271,
       0.34780489, 0.37945997, 0.37945997, 0.37945997, 0.37945997])

## Pykrige

In [9]:
from pykrige.ok3d import OrdinaryKriging3D

In [10]:
ok3d = OrdinaryKriging3D(x,y,z,cu_value, variogram_model=model)

In [52]:

k3d1, ss3d = ok3d.execute("grid", gridx, gridy, gridz)


## Comparacion

In [53]:
kriged_field


array([0.37945997, 0.37945997, 0.37945997, 0.39833957, 0.37957271,
       0.34780489, 0.37945997, 0.37945997, 0.37945997, 0.37945997])

In [55]:
k3d1[5][5][5]

0.347804894865737

La diferencia fundamental es que kriged field al parece solo predice los puntos en orden en el cual estan los gridx, gridy , gridz y no hace combinaciones, es decir 1 a 1, posicion a posicion. Si comparamos los resultados da exactamente el mismo valor.

Esto puede cambiar usando la instancia structured de gstool

In [103]:
kriged_field_2, variances_2 = krige.structured([gridx, gridy, gridz])


In [105]:
k3d1.data

array([[[0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997,
         0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997],
        [0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997,
         0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997],
        [0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997,
         0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997],
        [0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997,
         0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997],
        [0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997,
         0.57473387, 0.45363532, 0.37945997, 0.37945997, 0.37945997],
        [0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997,
         0.37950964, 0.40970048, 0.37945997, 0.37939069, 0.37945997],
        [0.37945997, 0.37945997, 0.37945997, 0.37945997, 0.37945997,
         0.37945997, 0.37945997, 0.40050631, 0.37945997, 0.37945997],
        [0.37945997, 0.3794

Como se puede apreciar en el siguiente cuadro de codigo, la diferencia entre las predicciones es minima

In [140]:
a = np.abs(kriged_field_2 - k3d1.data) > 0.5

a.any()

True

In [141]:
a[4]

array([[False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False,  True, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False]])

## Realizacion de Kriging con gstools sin hacer nada a los datos

In [1]:
import pandas as pd

In [2]:
df = pd.read_table('datos.txt', sep = ' ')
df.columns = [column.replace(" ", "_") for column in df.columns]
df.columns

Index(['id', 'x', 'y', 'z', 'tonn', 'blockvalue', 'destination', 'CU%',
       'process_profit'],
      dtype='object')

In [165]:
x = df['x'].to_numpy()
y = df['y'].to_numpy()
z = df['z'].to_numpy()
cu_value = df['CU%'].to_numpy()

In [166]:
len(x)

14153

In [167]:
# Crear un modelo de variograma esférico
model = gs.Spherical(dim=3, var=0.5, len_scale=1.5)

# Crear una instancia de Kriging Ordinario con el modelo
krige = gs.krige.Ordinary(model, [x, y, z], cu_value)

In [ ]:
# Realizar el Kriging en la grilla deseada
gridx = np.linspace(3, x.max(), 10)
gridy = np.linspace(2, y.max(), 10)
gridz = np.linspace(3, z.max(), 10)
# gridx, gridy, gridz = np.meshgrid(gridx, gridy, gridz)

# Obtener los valores estimados y las varianzas
kriged_field, variances = krige([gridx, gridy, gridz])

In [16]:
from sklearn.model_selection import train_test_split
import numpy as np
import gstools as gs

x = df['x'].to_numpy()
y = df['y'].to_numpy()
z = df['z'].to_numpy()
cu_value = df['CU%'].to_numpy()
features = np.column_stack((x, y, z))

X_train, X_test, y_train, y_test = train_test_split(features, cu_value, test_size=0.5, random_state=42)


In [17]:

model = gs.Spherical(dim=3, var=0.5, len_scale=1.5)
krige = gs.krige.Ordinary(model, X_train.T, y_train)  # Toma la transpuesta de X_train para ajustarse al formato esperado



In [9]:
X_test[0:4].T

array([[37, 30, 35, 45],
       [10, 18, 12, 30],
       [13, 10,  8, 10]], dtype=int64)

In [11]:
y_test[0:4]

array([0.8  , 1.241, 0.733, 0.   ])

In [18]:
kriged_values, variances = krige(X_test[0:4].T)


In [19]:
kriged_values

array([0.47710708, 0.57614163, 0.47283093, 0.25266763])

In [14]:
from sklearn.metrics import r2_score

score = r2_score(y_test[0:4], kriged_values)
score

-0.17759723290921148